<a href="https://colab.research.google.com/github/Lily-2002/Retrosynthetic_Planning/blob/main/test_bloomz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 31.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

In [5]:
from rdkit import Chem
def get_examples(n_examples,path):
    import numpy as np
    import random
    data  = pd.read_csv(path)
    examples = "Please predict the reactants of products.\n"
    unique_elements, indices = np.unique(np.array(list(data["retro_templates"])), return_index=True)
    # print(indices)
    indices = random.sample(list(indices),n_examples)
    # print(indices)
    for indice in indices:
        # print(indice)
        examples += "products:" + data.loc[indice,"products"] + " " + "reactants:" + data.loc[indice,"reactants"] + "\n"
# print(examples)
    return examples,indices

def extract_reactants_after_pattern(main_string, pattern_string,standard):
    print(f"pattern_string: {pattern_string}")
    print(f"main_string: {main_string}")
    index = main_string.find(pattern_string)
    reatants = main_string.find("reactants:",index + len(pattern_string))
    start = reatants + 11
    end = main_string.find("products:",index + len(pattern_string))
    print(end)
    if end != -1:
      reatants = main_string[start:end]
      # if reatants[-1] == "=":
      #   reatants = reatants[start:end - 1]
    else:
      reatants = main_string[start:]
    reatants = reatants.replace("\n","")
    # print("reatants")
    # print(reatants)
    # print("standard")
    # print(standard)
    if is_valid_smiles(reatants) == False:
      return 0
    else:
      mol = Chem.MolFromSmiles(reatants)
      # print(mol)
      mol_stan = Chem.MolFromSmiles(standard)
      # print(mol_stan)
      if Chem.MolToInchi(mol) == Chem.MolToInchi(mol_stan):
          return 1
      else:
          return 0
def eval(path,example,used_indices):
    data = pd.read_csv(path)
    acc = 0
    for index,row in data.iterrows():
        if index not in used_indices:
            prompt = example +"products:" + row["products"] + " " + "reactants:"
            # print(prompt)
            inputs = tokenizer(prompt, return_tensors='pt')
            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']
            outputs =  model.generate(
                            input_ids,
                            attention_mask=attention_mask,
                            top_p=0.9,
                            num_beams=8,
                            max_new_tokens=256,
                            num_return_sequences=8
                    )
            sum = 0
            for i, output in enumerate(outputs):
                generated_text = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
                a = extract_reactants_after_pattern(main_string = generated_text,pattern_string=row["products"],standard=row["reactants"])
                sum = sum + a
            sum = sum / 8
            acc = acc + sum
            print(f"ACC: {acc}")
    # print(acc)
    return acc
if __name__ == '__main__':
    import pandas as pd
    from transformers import AutoTokenizer, AutoModelForCausalLM
    tokenizer = AutoTokenizer.from_pretrained("/content/gdrive/MyDrive/BLOOMZ-560M")
    model = AutoModelForCausalLM.from_pretrained("/content/gdrive/MyDrive/BLOOMZ-560M")
    exp,idx  = get_examples(n_examples=4,path="/content/gdrive/MyDrive/syndata_for_enzyme.csv")
    eval(path="/content/gdrive/MyDrive/syndata_for_enzyme.csv",example=exp,used_indices=idx)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


pattern_string:  C[C@H](C(=O)O)C[C@H](C)OC=O                                              
main_string: Please predict the reactants of products.
products: O=P(O)(OCCCCCCCCCO)OCO                            reactants: O=P1(O)OCCCCCCCCCC(CO)O1
products: O=C(OO)OP(=O)(O)O  reactants: NC(=O)OP(=O)(O)O
products: O=C(OOP(=O)(O)O)O  reactants: NC(=O)OP(=O)(O)O
products: C[C@H](CCC(C)C)C(=O)OC=O                                                   reactants: C[C@H](CCC(C)C)C(=O)O
products: C[C@H](C(=O)O)C[C@H](C)OC=O                                               reactants: C[C@H](CCC(C)C)C(=O)O
-1
pattern_string:  C[C@H](C(=O)O)C[C@H](C)OC=O                                              
main_string: Please predict the reactants of products.
products: O=P(O)(OCCCCCCCCCO)OCO                            reactants: O=P1(O)OCCCCCCCCCC(CO)O1
products: O=C(OO)OP(=O)(O)O  reactants: NC(=O)OP(=O)(O)O
products: O=C(OOP(=O)(O)O)O  reactants: NC(=O)OP(=O)(O)O
products: C[C@H](CCC(C)C)C(=O)OC=O                

[11:13:39] WARNING: Accepted unusual valence(s): C(3)



pattern_string:  CCC[C@H](C)OC=O                                                            
main_string: Please predict the reactants of products.
products: O=P(O)(OCCCCCCCCCO)OCO                            reactants: O=P1(O)OCCCCCCCCCC(CO)O1
products: O=C(OO)OP(=O)(O)O  reactants: NC(=O)OP(=O)(O)O
products: O=C(OOP(=O)(O)O)O  reactants: NC(=O)OP(=O)(O)O
products: C[C@H](CCC(C)C)C(=O)OC=O                                                   reactants: C[C@H](CCC(C)C)C(=O)O
products: CCC[C@H](C)OC=O                                                             reactants: CCC[C@H](C)OC=O
-1
pattern_string:  CCC[C@H](C)OC=O                                                            
main_string: Please predict the reactants of products.
products: O=P(O)(OCCCCCCCCCO)OCO                            reactants: O=P1(O)OCCCCCCCCCC(CO)O1
products: O=C(OO)OP(=O)(O)O  reactants: NC(=O)OP(=O)(O)O
products: O=C(OOP(=O)(O)O)O  reactants: NC(=O)OP(=O)(O)O
products: C[C@H](CCC(C)C)C(=O)OC=O                

[11:14:12] SMILES Parse Error: syntax error while parsing: C[CCC(C)C)OC=O
[11:14:12] SMILES Parse Error: Failed parsing SMILES 'C[CCC(C)C)OC=O' for input: 'C[CCC(C)C)OC=O'
[11:14:12] SMILES Parse Error: syntax error while parsing: C[CCC(C)C)C(=O)O
[11:14:12] SMILES Parse Error: Failed parsing SMILES 'C[CCC(C)C)C(=O)O' for input: 'C[CCC(C)C)C(=O)O'
[11:14:12] WARNING: Omitted undefined stereo



...

[12:47:36] Explicit valence for atom # 7 O, 4, is greater than permitted
[12:47:36] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC[C@H](N)C(=O)O,
[12:47:36] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC[C@H](N)C(=O)O,' for input: 'NC(=O)NCCC[C@H](N)C(=O)O,'
[12:47:36] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC[C@H](N)C(=O)O,
[12:47:36] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC[C@H](N)C(=O)O,' for input: 'NC(=O)NCCC[C@H](N)C(=O)O,'
[12:47:36] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC[C@H](N)C(=O)O,
[12:47:36] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC[C@H](N)C(=O)O,' for input: 'NC(=O)NCCC[C@H](N)C(=O)O,'
[12:47:36] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC[C@H](N)C(=O)O,
[12:47:36] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC[C@H](N)C(=O)O,' for input: 'NC(=O)NCCC[C@H](N)C(=O)O,'


pattern_string:  NC(=O)NCCC[C@H](N)C(=O)O, N[C@@H](CC(=O)O)C(=O)O 
main_string: Please predict the reactants of products.
products: O=P(O)(OCCCCCCCCCO)OCO                            reactants: O=P1(O)OCCCCCCCCCC(CO)O1
products: O=C(OO)OP(=O)(O)O  reactants: NC(=O)OP(=O)(O)O
products: O=C(OOP(=O)(O)O)O  reactants: NC(=O)OP(=O)(O)O
products: C[C@H](CCC(C)C)C(=O)OC=O                                                   reactants: C[C@H](CCC(C)C)C(=O)O
products: NC(=O)NCCC[C@H](N)C(=O)O, N[C@@H](CC(=O)O)C(=O)O  reactants: NC(=O)NCCC[C@H](N)C(=O)O
-1
pattern_string:  NC(=O)NCCC[C@H](N)C(=O)O, N[C@@H](CC(=O)O)C(=O)O 
main_string: Please predict the reactants of products.
products: O=P(O)(OCCCCCCCCCO)OCO                            reactants: O=P1(O)OCCCCCCCCCC(CO)O1
products: O=C(OO)OP(=O)(O)O  reactants: NC(=O)OP(=O)(O)O
products: O=C(OOP(=O)(O)O)O  reactants: NC(=O)OP(=O)(O)O
products: C[C@H](CCC(C)C)C(=O)OC=O                                                   reactants: C[C@H](CCC(C)C)C(=O)O
p

[12:48:27] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC[C@H](N)C(=O)O,
[12:48:27] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC[C@H](N)C(=O)O,' for input: 'NC(=O)NCCC[C@H](N)C(=O)O,'
[12:48:27] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC[C@H](N)C(=O)O,
[12:48:27] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC[C@H](N)C(=O)O,' for input: 'NC(=O)NCCC[C@H](N)C(=O)O,'
[12:48:27] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC(C@H)C(=O)O
[12:48:27] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC(C@H)C(=O)O' for input: 'NC(=O)NCCC(C@H)C(=O)O'
[12:48:27] WARNING: Omitted undefined stereo



pattern_string:  NC(=O)NCCC[C@H](N)C(=O)O, N[C@@H](CCC(=O)O)C(=O)O 
main_string: Please predict the reactants of products.
products: O=P(O)(OCCCCCCCCCO)OCO                            reactants: O=P1(O)OCCCCCCCCCC(CO)O1
products: O=C(OO)OP(=O)(O)O  reactants: NC(=O)OP(=O)(O)O
products: O=C(OOP(=O)(O)O)O  reactants: NC(=O)OP(=O)(O)O
products: C[C@H](CCC(C)C)C(=O)OC=O                                                   reactants: C[C@H](CCC(C)C)C(=O)O
products: NC(=O)NCCC[C@H](N)C(=O)O, N[C@@H](CCC(=O)O)C(=O)O  reactants: NC(=O)NCCC[C@H](N)C(=O)O
-1
pattern_string:  NC(=O)NCCC[C@H](N)C(=O)O, N[C@@H](CCC(=O)O)C(=O)O 
main_string: Please predict the reactants of products.
products: O=P(O)(OCCCCCCCCCO)OCO                            reactants: O=P1(O)OCCCCCCCCCC(CO)O1
products: O=C(OO)OP(=O)(O)O  reactants: NC(=O)OP(=O)(O)O
products: O=C(OOP(=O)(O)O)O  reactants: NC(=O)OP(=O)(O)O
products: C[C@H](CCC(C)C)C(=O)OC=O                                                   reactants: C[C@H](CCC(C)C)C(=O)

[12:49:19] Explicit valence for atom # 7 O, 4, is greater than permitted
[12:49:19] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC[C@H](N)C(=O)O,
[12:49:19] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC[C@H](N)C(=O)O,' for input: 'NC(=O)NCCC[C@H](N)C(=O)O,'
[12:49:19] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC(C@H)C(=O)O
[12:49:19] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC(C@H)C(=O)O' for input: 'NC(=O)NCCC(C@H)C(=O)O'
[12:49:19] SMILES Parse Error: syntax error while parsing: NC(=O)NCCC[C@H](N)C(=O)O,
[12:49:19] SMILES Parse Error: Failed parsing SMILES 'NC(=O)NCCC[C@H](N)C(=O)O,' for input: 'NC(=O)NCCC[C@H](N)C(=O)O,'


In [6]:
import pandas as pd
data = pd.read_csv("/content/gdrive/MyDrive/syndata_for_enzyme.csv")
result = 11.25 / len(list(data["reactants"]))
print(result)

0.10817307692307693


In [ ]:
from rdkit import Chem



# 示例用法
smiles1 = "C[C@H](OC=O)(O)O"
smiles2 = "ABC"

print(f"Is '{smiles1}' a valid SMILES? {is_valid_smiles(smiles1)}")
print(f"Is '{smiles2}' a valid SMILES? {is_valid_smiles(smiles2)}")


Is 'C[C@H](OC=O)(O)O' a valid SMILES? False
Is 'ABC' a valid SMILES? False


[02:55:01] Explicit valence for atom # 1 C, 5, is greater than permitted
[02:55:01] SMILES Parse Error: syntax error while parsing: ABC
[02:55:01] SMILES Parse Error: Failed parsing SMILES 'ABC' for input: 'ABC'
